# Part II – Feedforward NNs, Autoencoders and Convolutional NNs

## This is a modification of previous notebook. Modifications: 

- Based on: `ML-ac-ficha6-PII-2.2.ipynb`

**Changes:**
    - Applying it for decision trees.

**Data import:**

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) =mnist.load_data()
# Normalize the images to the range [0, 1]
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
# Convert the labels to one-hot encoded vectors
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

**Neural Network:**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape

In [3]:

# ENCODER #
# Define input layer
encoder_input = Input(shape=(28, 28, 1))
# Define the decoder layers
z = encoder_input
z = Flatten()(z)
z = Dense(64, activation='relu')(z)
z = Dense(10, activation='relu')(z)
encoder_layer = z 
encoder = Model(encoder_input, encoder_layer)

# DECODER #
## Input
decoder_input = Input(shape=encoder_layer.shape[1:])
##  Layers
y = decoder_input
y = Dense(64, activation='relu')(y)
y = Dense(784, activation='relu')(y)
y = Reshape((28, 28, 1))(y)
decoder_layer = y
## Model
decoder = Model(decoder_input, decoder_layer)

# AUTOENCODER #
## Model
autoencoder = Model(encoder_input, decoder(encoder(encoder_input)))
model = autoencoder
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_images, epochs=5, batch_size=64, validation_split=0.2)


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 model (Functional)          (None, 10)                50890     
                                                                 
 model_1 (Functional)        (None, 28, 28, 1)         51664     
                                                                 
Total params: 102554 (400.60 KB)
Trainable params: 102554 (400.60 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Epoch 1/5


750/750 [==============================] - 6s 7ms/step - loss: 0.0405 - accuracy: 0.8048 - val_loss: 0.0329 - val_accuracy: 0.8083
Epoch 2/5
750/750 [==============================] - 5s 7ms/step - loss: 0.0312 - accuracy: 0.8077 - val_loss: 0.0300 - val_accuracy: 0.80

# Decision Tree Part

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

**Load features:**

In [16]:
# Extract features using the encoder model
train_features = encoder.predict(train_images)
test_features = encoder.predict(test_images)

313/313 [==============================] - 1s 2ms/step


In [22]:
# Claryifying the shape of the original data.
print(f"Shape of the original images data: {train_images.shape}")
# Claryifying the shape of the original labels.
print(f"Shape of the original labels data: {train_labels.shape}")
# Claryifying the shape of the original data.
print(f"Shape of the encoded image data: {train_features.shape}")

Shape of the original images data: (60000, 28, 28, 1)
Shape of the original labels data: (60000, 10)
Shape of the encoded image data: (60000, 10)


Remember that we one-hot encoded our data. Therefore now we undo one-hot encodding, which we do by using the np.argmax function.

Details: Before our labels were something like `[ 9 9 0 1 7 2 ...]`

In [35]:
# Undoes the one-hot encoding: 
train_labels_int = np.argmax(train_labels, axis=1)
test_labels_int = np.argmax(test_labels, axis=1)
# print(train_labels_int)
# print(test_labels_int)
# print(train_labels.shape)
# print(train_labels_int.shape)

# #To confirme the one-hot encoding is being undone properly
# print(train_labels[1:10])
# #The below should only contain values in the range 0-9:
# print(train_labels_int)


[5 0 4 ... 5 6 8]
[7 2 1 ... 4 5 6]
(60000, 10)
(60000,)


Confiring that the one-hot doing an undoing is working properly:

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [24]:
# Initialize and train the Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(train_features, train_labels_int)

# Predict on the test data
test_predictions = decision_tree.predict(test_features)

# Evaluate the classifier
accuracy = accuracy_score(test_labels_int, test_predictions)
report = classification_report(test_labels_int, test_predictions)

print(f"Decision Tree Classifier Accuracy: {accuracy}")
print("Classification Report:\n", report)

Decision Tree Classifier Accuracy: 0.8247
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.87       980
           1       0.96      0.96      0.96      1135
           2       0.84      0.84      0.84      1032
           3       0.79      0.81      0.80      1010
           4       0.79      0.78      0.78       982
           5       0.76      0.74      0.75       892
           6       0.91      0.90      0.90       958
           7       0.85      0.85      0.85      1028
           8       0.75      0.75      0.75       974
           9       0.70      0.73      0.72      1009

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.83      0.82      0.82     10000



More.

THIS TREE IS GIANT, depth 32????

In [36]:
# After you have trained your decision tree:
number_of_nodes = decision_tree.tree_.node_count
depth_of_tree = decision_tree.tree_.max_depth
number_of_leaves = decision_tree.get_n_leaves()

print(f"Number of nodes in the tree: {number_of_nodes}")
print(f"Depth of the tree: {depth_of_tree}")
print(f"Number of leaf nodes: {number_of_leaves}")


Number of nodes in the tree: 13493
Depth of the tree: 32
Number of leaf nodes: 6747
